In [1]:
from dask.distributed import LocalCluster, Client


In [2]:
# if you are copy pasting this from the book, make sure 
# the address is the same that the instructor gave in the lesson!
address = 'tcp://127.0.0.1:34217'
client = Client(address)
client

<Client: 'tcp://127.0.0.1:34217' processes=70 threads=140, memory=220.10 GiB>

In [2]:
import os 
import urllib

import dask.dataframe as dd

## Import data 

In [4]:
url = 'https://arcticdata.io/metacat/d1/mn/v2/object/urn%3Auuid%3A27e4043d-75eb-4c4f-9427-0d442526c154'

msg = urllib.request.urlretrieve(url, "dg_soil_moisture.csv")

In [8]:
import pandas as pd

# pd.read_csv('dg_soil_moisture.csv', nrows=10, encoding='ISO-8859-1')
fp = 'dg_soil_moisture.csv'
df = dd.read_csv(fp, blocksize='20MB', encoding='ISO-8859-1')

In [9]:
df

,timestamp,year,doy,hour,minute,site,logger,port,sensor,sensorZ,m_soil,unit
npartitions=6,,,,,,,,,,,,
,string,int64,int64,int64,int64,string,string,string,string,int64,float64,string
,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...


In [10]:
df.head()

,timestamp,year,doy,hour,minute,site,logger,port,sensor,sensorZ,m_soil,unit
0,2014-07-07 16:30:00,2014,188,16,30,MDF1,MDF1met,Port 3,5TM Moisture/Temp,-6,0.273,m³/m³ VWC
1,2014-07-07 16:30:00,2014,188,16,30,MDF1,MDF1met,Port 4,5TM Moisture/Temp,-11,0.345,m³/m³ VWC
2,2014-07-07 17:00:00,2014,188,17,0,LDF2,LDF2met,Port 3,5TM Moisture/Temp,-8,0.308,m³/m³ VWC
3,2014-07-07 17:00:00,2014,188,17,0,LDF2,LDF2met,Port 4,5TM Moisture/Temp,-13,0.325,m³/m³ VWC
4,2014-07-07 17:00:00,2014,188,17,0,MDF1,MDF1met,Port 3,5TM Moisture/Temp,-6,0.283,m³/m³ VWC


### Note that each 

In [12]:
type(df)

dask.dataframe.dask_expr._collection.DataFrame

In [11]:
type(df.head())

pandas.core.frame.DataFrame

In [13]:
averages = df.groupby('year').mean(numeric_only=True)
averages

,doy,hour,minute,sensorZ,m_soil
npartitions=1,,,,,
,float64,float64,float64,float64,float64
,...,...,...,...,...


In [14]:
averages.compute()

,doy,hour,minute,sensorZ,m_soil
year,,,,,
2014,276.852636,11.513992,15.001123,-11.998332,0.273744
2015,186.720383,11.500723,15.000000,-13.047899,0.263738
2016,183.497453,11.499777,14.999858,-15.000009,0.293595
2017,181.414843,11.499381,15.000144,-14.999981,0.266121
2018,201.824077,11.500796,15.000356,-15.435365,0.282395
2019,173.693311,11.498577,15.000000,-15.124516,0.222193
2020,138.806679,11.489825,14.999322,-15.200054,0.252467


# Dask arrays

In [15]:
import numpy as np
import dask.array as da

In [16]:
data = np.arange(100_000).reshape(200,500)
data

array([[    0,     1,     2, ...,   497,   498,   499],
       [  500,   501,   502, ...,   997,   998,   999],
       [ 1000,  1001,  1002, ...,  1497,  1498,  1499],
       ...,
       [98500, 98501, 98502, ..., 98997, 98998, 98999],
       [99000, 99001, 99002, ..., 99497, 99498, 99499],
       [99500, 99501, 99502, ..., 99997, 99998, 99999]], shape=(200, 500))

In [17]:
a = da.from_array(data, chunks = (100,100))
a

dask.array<array, shape=(200, 500), dtype=int64, chunksize=(100, 100), chunktype=numpy.ndarray>

In [ ]:
# this is "lazy," it doesn't actually do anything yet
a.mean()

dask.array<mean_agg-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

In [ ]:
# this actually makes the computation
a.mean().compute()


np.float64(49999.5)

### reading in a tif file

In [20]:
# download red band
url = 'https://arcticdata.io/metacat/d1/mn/v2/object/urn%3Auuid%3Aac25a399-b174-41c1-b6d3-09974b161e5a'
msg = urllib.request.urlretrieve(url, "RU_ANS_TR2_FL005M_red.tif")


# download nir band
url = 'https://arcticdata.io/metacat/d1/mn/v2/object/urn%3Auuid%3A1762205e-c505-450d-90ed-d4f3e4c302a7'

msg = urllib.request.urlretrieve(url, "RU_ANS_TR2_FL005M_nir.tif")

In [3]:
import rioxarray as rioxr

In [4]:
# read in the file
fp_red = os.path.join(os.getcwd(),"RU_ANS_TR2_FL005M_red.tif")
red = rioxr.open_rasterio(fp_red, chunks = '15MB')
red

<xarray.DataArray (band: 1, y: 3499, x: 7443)> Size: 104MB
dask.array<open_rasterio-96e6eb3efbca08b3bce2c59d10d5e1fd<this-array>, shape=(1, 3499, 7443), dtype=float32, chunksize=(1, 1936, 1936), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 60kB 6.22e+05 6.22e+05 ... 6.224e+05 6.224e+05
  * y            (y) float64 28kB 7.525e+06 7.525e+06 ... 7.525e+06 7.525e+06
    spatial_ref  int64 8B 0
Attributes:
    TIFFTAG_SOFTWARE:  pix4dmapper
    AREA_OR_POINT:     Area
    _FillValue:        -10000.0
    scale_factor:      1.0
    add_offset:        0.0

In [5]:
# getting rid of unnecessary dimension
red = red.squeeze()
red

<xarray.DataArray (y: 3499, x: 7443)> Size: 104MB
dask.array<getitem, shape=(3499, 7443), dtype=float32, chunksize=(1936, 1936), chunktype=numpy.ndarray>
Coordinates:
    band         int64 8B 1
  * x            (x) float64 60kB 6.22e+05 6.22e+05 ... 6.224e+05 6.224e+05
  * y            (y) float64 28kB 7.525e+06 7.525e+06 ... 7.525e+06 7.525e+06
    spatial_ref  int64 8B 0
Attributes:
    TIFFTAG_SOFTWARE:  pix4dmapper
    AREA_OR_POINT:     Area
    _FillValue:        -10000.0
    scale_factor:      1.0
    add_offset:        0.0

In [6]:
red

<xarray.DataArray (y: 3499, x: 7443)> Size: 104MB
dask.array<getitem, shape=(3499, 7443), dtype=float32, chunksize=(1936, 1936), chunktype=numpy.ndarray>
Coordinates:
    band         int64 8B 1
  * x            (x) float64 60kB 6.22e+05 6.22e+05 ... 6.224e+05 6.224e+05
  * y            (y) float64 28kB 7.525e+06 7.525e+06 ... 7.525e+06 7.525e+06
    spatial_ref  int64 8B 0
Attributes:
    TIFFTAG_SOFTWARE:  pix4dmapper
    AREA_OR_POINT:     Area
    _FillValue:        -10000.0
    scale_factor:      1.0
    add_offset:        0.0

In [7]:
# open data
fp_nir = os.path.join(os.getcwd(),"RU_ANS_TR2_FL005M_nir.tif")
nir = rioxr.open_rasterio(fp_nir, chunks = '15MB')

#squeeze
nir = nir.squeeze()

In [8]:
ndvi = (nir - red) / (nir + red)
ndvi

<xarray.DataArray (y: 3499, x: 7443)> Size: 104MB
dask.array<truediv, shape=(3499, 7443), dtype=float32, chunksize=(1936, 1936), chunktype=numpy.ndarray>
Coordinates:
    band         int64 8B 1
  * x            (x) float64 60kB 6.22e+05 6.22e+05 ... 6.224e+05 6.224e+05
  * y            (y) float64 28kB 7.525e+06 7.525e+06 ... 7.525e+06 7.525e+06
    spatial_ref  int64 8B 0

In [9]:
ndvi_values = ndvi.compute()

In [11]:
nir.values

array([[-10000., -10000., -10000., ..., -10000., -10000., -10000.],
       [-10000., -10000., -10000., ..., -10000., -10000., -10000.],
       [-10000., -10000., -10000., ..., -10000., -10000., -10000.],
       ...,
       [-10000., -10000., -10000., ..., -10000., -10000., -10000.],
       [-10000., -10000., -10000., ..., -10000., -10000., -10000.],
       [-10000., -10000., -10000., ..., -10000., -10000., -10000.]],
      shape=(3499, 7443), dtype=float32)

In [12]:
nir.data

dask.array<getitem, shape=(3499, 7443), dtype=float32, chunksize=(1936, 1936), chunktype=numpy.ndarray>

In [10]:
ndvi_values

<xarray.DataArray (y: 3499, x: 7443)> Size: 104MB
array([[-0., -0., -0., ..., -0., -0., -0.],
       [-0., -0., -0., ..., -0., -0., -0.],
       [-0., -0., -0., ..., -0., -0., -0.],
       ...,
       [-0., -0., -0., ..., -0., -0., -0.],
       [-0., -0., -0., ..., -0., -0., -0.],
       [-0., -0., -0., ..., -0., -0., -0.]],
      shape=(3499, 7443), dtype=float32)
Coordinates:
    band         int64 8B 1
  * x            (x) float64 60kB 6.22e+05 6.22e+05 ... 6.224e+05 6.224e+05
  * y            (y) float64 28kB 7.525e+06 7.525e+06 ... 7.525e+06 7.525e+06
    spatial_ref  int64 8B 0